In [1]:
from datasets import load_dataset, Dataset
import spacy
from tqdm.auto import tqdm
from collections import defaultdict
from nltk import sent_tokenize
import joblib
#data = load_dataset("Seongill/nq_squad")["train"]

In [274]:
data.select(range(10))

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'masked_query', 'query_embedding'],
    num_rows: 10
})

In [282]:
sents = []
for row in tqdm(data):
    context, answers = row["context"], row["answers"]
    sentences = sent_tokenize(context)
    cnt = 0
    is_sent = False
    for sent in sentences:
        if is_sent:
            break
        for ans in answers["text"]:
            if is_sent:
                break
            if ans in sent:
                is_sent = True
                sents.append(sent)
    if not is_sent:
        sents.append("")
print(len(sents), len(data))

  0%|          | 0/98169 [00:00<?, ?it/s]

98169 98169


In [285]:
cnt = 0
for row, sent in zip(tqdm(data), sents):
    answers = row["answers"]["text"]
    output = any([ans in sent for ans in answers])
    if not output:
        cnt+=1

  0%|          | 0/98169 [00:00<?, ?it/s]

In [286]:
print(cnt)

223


In [239]:
BATCH_SIZE = 8000
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")

In [281]:
docs = []
for i in tqdm(range(0, len(sents), BATCH_SIZE)):
    batch = sents[i:i+BATCH_SIZE]
    docs.extend(list(nlp.pipe(batch)))

  0%|          | 0/13 [00:00<?, ?it/s]

In [294]:
cnt = 0
result, ent_type = [], []
for doc, answers in zip(tqdm(docs),data["answers"]):
    is_ne = False
    for ent in doc.ents:
        if ent.text == answers["text"][0]:
            is_ne = True
            ent_type.append(ent.label_)
            break
    if not is_ne:
        ent_type.append("")
    result.append(is_ne)

  0%|          | 0/98169 [00:00<?, ?it/s]

In [359]:
df = pd.DataFrame(data)

In [360]:
df["answer_sentence"] = sents
df["is_named_entity"] = is_ne
df["ent_type"] = ent_type
df = df.drop_duplicates(subset=['question'], keep='first')

In [361]:
df = df[df["ent_type"] != ""]
df.groupby("ent_type").count().iloc[:, 0]

ent_type
CARDINAL       3982
DATE           8127
EVENT           479
FAC            1101
GPE            3716
LANGUAGE        338
LAW             192
LOC             797
MONEY           433
NORP           1571
ORDINAL         211
ORG            3695
PERCENT         984
PERSON         6856
PRODUCT         467
QUANTITY        492
TIME            107
WORK_OF_ART    1192
Name: id, dtype: int64

In [362]:
len(df)

34740

In [ ]:
ent_to_answer = defaultdict(list)
answers = [row["text"][0] for row in df["answers"].values]
ents = df["ent_type"].values.tolist()
for ans, ent in zip(answers, ents):
    ent_to_answer[ent].append(ans)
ent_to_answer_unique = dict()
for k,v in ent_to_answer.items():
    ent_to_answer_unique[k] = list(set(v))
cnt = 0
for k,v in ent_to_answer_unique.items():
    print(f'{k} {len(v)}')
    cnt += len(v)
print(cnt)

In [365]:
df["num_words"] = df["context"].apply(lambda x: len(x.split()))
df = df[df.num_words < 150]

In [363]:
from datasets import Dataset
import pandas as pd
dataset_filtered = Dataset.from_pandas(df)
print(dataset_filtered)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'masked_query', 'query_embedding', 'answer_sentence', 'is_named_entity', 'ent_type', '__index_level_0__'],
    num_rows: 34740
})


In [364]:
dataset_filtered.push_to_hub("squad_conflict_v2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

In [378]:
df["rewritten_context"] = df["context"].apply(lambda x: old_new_map.get(x, -1))
df = df[df["rewritten_context"] != -1]
has_answers = []
new_ans = []
for r in df.iterrows():
    row = r[1]
    answer = row["answers"]["text"][0]
    new_ans.append(answer)
    has_answer = answer in row["rewritten_context"]
    has_answers.append(has_answer)
df["has_answer"] = has_answers
df["answer"] = new_ans

In [379]:
len(df)

26559

In [ ]:
old_new_map

In [83]:
context_answer_map = defaultdict(list)
for row in df.iterrows():
    c, a = row[1]["context"], row[1]["answers"]["text"]
    context_answer_map[c].extend(a)

In [86]:
print(len(context_answer_map.keys()))

14263


In [90]:
print(context_answer_map[list(context_answer_map.keys())[0]])

['a copper statue of Christ', 'the Main Building', 'a Marian place of prayer and reflection']


In [89]:
list(context_answer_map.keys())[0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [69]:
ent_answer_map_unique = dict()
for k, v in ent_answer_map.items():
    ent_answer_map_unique[k] = list(set(v))

In [101]:
ctxs, answers = list(context_answer_map.keys()), list(context_answer_map.values())

In [108]:
def make_prompt(context, answers):
    return f"Rewrite the following ariticle, maintaining its original meaning. Do not add any new information. Keep the specified words unchanged.\nWords to Keep Unchanged: {', '.join(answers)}\n\nOriginal Article: {context}\nRewritten Article:"

In [113]:
prompts = [make_prompt(c,a) for c, a in zip(ctxs, answers)]
len(prompts)

14263

In [61]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.environ.get("OPENAI_APIKEY"))

In [172]:
len(ctxs)

14263

In [173]:
BATCH_SIZE = 20
preds = []
recall = []
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch = prompts[i:i+BATCH_SIZE]
    batch_ans = answers[i:i+BATCH_SIZE]
    responses = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=batch,
        seed=42,
        max_tokens=1024
    )
    for res, answer in zip(responses.choices, batch_ans):
        pred = res.text.strip()
        recall.append(sum([ans in pred for ans in answer])/len(answer))
        preds.append(pred)

  0%|          | 0/714 [00:00<?, ?it/s]

In [174]:
import joblib
joblib.dump(preds, "gpt_output.joblib")

['gpt_output.joblib']

In [176]:
old_new_map = dict()
for ctx, pred in zip(ctxs, preds):
    old_new_map[ctx] = pred

In [181]:
df["rewritten_context"] = df["context"].apply(lambda x: old_new_map[x])
has_answers = []
new_ans = []
for r in df.iterrows():
    row = r[1]
    answer = row["answers"]["text"][0]
    new_ans.append(answer)
    has_answer = answer in row["rewritten_context"]
    has_answers.append(has_answer)
df["has_answer"] = has_answers
df["answer"] = new_ans

In [70]:
for k, v in ent_answer_map_unique.items():
    print(f"{k if k != '' else None} : {len(v)}")

None : 36204
PERSON : 13847
FAC : 949
NORP : 1986
DATE : 4072
CARDINAL : 3199
GPE : 2442
ORG : 4424
WORK_OF_ART : 433
PERCENT : 750
QUANTITY : 781
ORDINAL : 381
MONEY : 768
EVENT : 462
LANGUAGE : 132
PRODUCT : 254
LOC : 931
LAW : 226
TIME : 228


In [8]:
print(len(data["train"]["question"]))
print(len(list(set(data["train"]["question"]))))

98169
97888


In [4]:
def check_retrieval(answers, contexts):
    has_answers = [c["hasanswer"] for c in contexts]

In [17]:
recall = 0
for row in data["train"]:
    ctxs = row["ctxs"]
    recall += int(any([c["hasanswer"] for c in ctxs]))
print(recall / len(data["train"]))

0.6829798123400626


In [18]:
recall = 0
for row in data["test"]:
    ctxs = row["ctxs"]
    recall += int(any([c["hasanswer"] for c in ctxs]))
print(recall / len(data["test"]))

0.7096952908587257


In [ ]:
!python -m spacy download en_core_web_lg

In [385]:
import spacy
nlp = spacy.load("en_core_web_lg")  # 여기서 "en_core_web_sm"은 선택한 모델에 따라 달라질 수 있습니다.

In [5]:
#joblib.dump(ent_to_answer_unique, "ent_to_answer_unique.joblib")
ent_to_answer_unique = joblib.load("ent_to_answer_unique.joblib")
data = load_dataset("Seongill/squad_conflict_v2_under_150")["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/26799 [00:00<?, ? examples/s]

20208

In [13]:
from collections import Counter
counter = Counter(data["ent_type"])
counter

Counter({'DATE': 6224,
         'PERSON': 4933,
         'CARDINAL': 3110,
         'ORG': 2989,
         'GPE': 2960,
         'NORP': 1242,
         'FAC': 878,
         'WORK_OF_ART': 850,
         'PERCENT': 806,
         'LOC': 678,
         'QUANTITY': 391,
         'PRODUCT': 363,
         'EVENT': 361,
         'MONEY': 347,
         'LANGUAGE': 274,
         'ORDINAL': 174,
         'LAW': 144,
         'TIME': 75})

In [15]:
for k, v in ent_to_answer_unique.items():
    print(f"{k} : {len(v)}")

FAC : 933
DATE : 3358
CARDINAL : 1508
GPE : 1562
ORG : 2900
PERSON : 4961
WORK_OF_ART : 956
PERCENT : 581
QUANTITY : 463
ORDINAL : 66
MONEY : 368
NORP : 835
EVENT : 361
LANGUAGE : 114
PRODUCT : 414
LOC : 563
LAW : 171
TIME : 94


In [16]:
ent_to_answer_unique = joblib.load("ent_to_answer_unique.joblib")
unique_mentions = []
mention_to_ner = dict()
for k,v in ent_to_answer_unique.items():
    if k == "":
        print(len(v))
        continue
    else:
        unique_mentions.extend(v)
for k, v in ent_to_answer_unique.items():
    if k == "":
        continue
    for mention in v:
        mention_to_ner[mention] = k
print(len(unique_mentions), len(mention_to_ner.keys()))

20208 19912


In [19]:
for unique_mention in unique_mentions:
    mention_to_ner[unique_mention]

FAC


In [17]:
import spacy
nlp = spacy.load("en_core_web_lg")
spacy.prefer_gpu()

True

In [20]:
ent_type_vector_map = defaultdict(list)
embeds = []
for i in tqdm(range(0, len(unique_mentions), 2000)):
    batch = unique_mentions[i:i+2000]
    for mention, _doc in zip(batch, list(nlp.pipe(batch))):
        embeds.append(_doc.vector)
        ent_type_vector_map[mention_to_ner[mention]].append(_doc.vector)
len(embeds)

  0%|          | 0/11 [00:00<?, ?it/s]

20208

In [27]:
import numpy as np
import pandas as pd
for k, v in ent_type_vector_map.items():
    ent_type_vector_map[k] = np.array(ent_type_vector_map[k])

In [28]:
entity_vector_map = dict()
for ent, vec in zip(unique_mentions, embeds):
    entity_vector_map[ent] = vec
# entity 별로 vector 인덱스를 만들고, 거기서 FAISS similarity search!

In [29]:
df = pd.DataFrame(data)

In [32]:
df = df.drop("__index_level_0__", axis=1)
df["answer"] = df.answers.apply(lambda x: x["text"][0])

In [35]:
df["random_answer"] = df["answer"].apply(lambda x: find_random_entity(mention_to_ner[x], x))

In [42]:
res = []
for row in tqdm(df.iterrows()):
    res.append(find_non_identical_random_entity(mention_to_ner[row[1]["answer"]], row[1]["answer"]))

0it [00:00, ?it/s]

/tmp/ipykernel_918737/2546982892.py:6: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))


In [43]:
df["similar_answer"] = res

In [46]:
all_df = load_dataset("Seongill/squad_conflict_all")['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42943 [00:00<?, ? examples/s]

In [47]:
old_new_map = dict()
for row in all_df:
    ctx, new_ctx = row["context"], row["rewritten_context"]
    old_new_map[ctx] = new_ctx

In [64]:
df["rewritten_context"] = df.context.apply(lambda x: old_new_map.get(x))

In [66]:
has_answers = []
for r in df.iterrows():
    row = r[1]
    answer = row["answer"]
    has_answer = answer in row["rewritten_context"]
    has_answers.append(has_answer)
df["has_answer"] = has_answers

In [ ]:
df[df.has_answer == True]

In [72]:
df = df[df.has_answer == True]
df = df.drop(["is_named_entity", "num_words", "answer_sentence"], axis=1)

In [73]:
df

,id,title,context,question,answers,masked_query,query_embedding,ent_type,answer,random_answer,similar_answer,rewritten_context,has_answer
0,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'answer_start': [279], 'text': ['the Main Bui...",[MASK] at [MASK] is beside to which structure?,"[0.3204971253871918, 0.20272424817085266, 0.22...",FAC,the Main Building,U.S. 1,the Royal Mews,"With its Catholic character, the school's arch...",True
1,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'answer_start': [119], 'text': ['Rome']}",Where is the headquarters of [MASK]?,"[0.3285101652145386, 0.22826211154460907, 0.12...",GPE,Rome,Baden-Württemberg,Antioch,"Moreau Seminary, located on the campus borderi...",True
2,5733bed24776f41900661189,University_of_Notre_Dame,The university is the major seat of the Congre...,What is the primary seminary of the Congregati...,"{'answer_start': [145], 'text': ['Moreau Semin...",What is the primary seminary of [MASK]?,"[0.1649821400642395, 0.1607779562473297, 0.091...",ORG,Moreau Seminary,The Entertainment Channel,Hofstra University,"Moreau Seminary, located on the campus borderi...",True
3,5733bed24776f4190066118a,University_of_Notre_Dame,The university is the major seat of the Congre...,What is the oldest structure at Notre Dame?,"{'answer_start': [234], 'text': ['Old College']}",What is the oldest structure at [MASK]?,"[-0.02514331042766571, 0.3645743131637573, -0....",ORG,Old College,the Insurance Institute for Highway Safety,Eton College,"Moreau Seminary, located on the campus borderi...",True
4,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'answer_start': [487], 'text': ['eight']}",How many BS level degrees are offered in [MASK...,"[0.6788813471794128, 0.5213160514831543, 0.146...",CARDINAL,eight,2.1 million,over twenty,The College of Engineering was founded in 1920...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26794,57378e311c456719005744b1,Force,The origin of electric and magnetic fields wou...,How many scalar equations were formed into a s...,"{'answer_start': [159, 159, 159, 159], 'text':...",How many scalar equations were formed into a s...,"[0.3210879862308502, 0.09044459462165833, 0.41...",CARDINAL,20,sextuple,5 to 7,James Clerk Maxwell was responsible for fully ...,True
26795,57378e311c456719005744b2,Force,The origin of electric and magnetic fields wou...,How many vector equations did Heaviside and Gi...,"{'answer_start': [215, 215, 215, 215], 'text':...",How many vector equations did [MASK] and Gibbs...,"[-0.013793256133794785, -0.11560852080583572, ...",CARDINAL,4,"approximately 119,000",5 to 7,James Clerk Maxwell was responsible for fully ...,True
26796,57378e311c456719005744b3,Force,The origin of electric and magnetic fields wou...,Who discovered that magnetic and electric coul...,"{'answer_start': [444, 88, 444, 444], 'text': ...",Who discovered that magnetic and electric coul...,"[0.1881973296403885, -0.14856640994548798, -0....",PERSON,Maxwell,Daniel Robbins,Thomas Edward Gordon,James Clerk Maxwell was responsible for fully ...,True
26797,57379829c3c5551400e51f41,Force,The weak force is due to the exchange of the h...,At what temperature do weak and electromagneti...,"{'answer_start': [514, 501, 528, 501], 'text':...",At what temperature do weak and electromagneti...,"[0.15145105123519897, -0.07466116547584534, 0....",QUANTITY,approximately 1015 kelvins,50 m,267 kilometers,The heavy W and Z bosons are responsible for t...,True


In [56]:
ctx_to_answer = defaultdict(list)
ctxs, answers = [], []
for row in df[df["rewritten_context"].isnull()].iterrows():
    ctxs.append(row[1]["context"])
    answers.append(row[1]["answer"])
    ctx_to_answer[row[1]["context"]].append(row[1]["answer"])

In [59]:
def make_prompt(context, answers):
    return f"Rewrite the following ariticle, maintaining its original meaning. Do not add any new information. Keep the specified words unchanged.\nWords to Keep Unchanged: {', '.join(answers)}\n\nOriginal Article: {context}\nRewritten Article:"

In [60]:
prompts = [make_prompt(k, v) for k, v in ctx_to_answer.items()]
len(prompts)

185

In [62]:
BATCH_SIZE = 20
preds = []
recall = []
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch = prompts[i:i+BATCH_SIZE]
    batch_ans = answers[i:i+BATCH_SIZE]
    responses = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=batch,
        seed=42,
        max_tokens=1024
    )
    for res, answer in zip(responses.choices, batch_ans):
        pred = res.text.strip()
        recall.append(sum([ans in pred for ans in answer])/len(answer))
        preds.append(pred)

  0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
for pred, ctx in zip(preds, ctx_to_answer.keys()):
    old_new_map.update({ctx:pred})

In [77]:
final = Dataset.from_pandas(df)
#

In [78]:
final.push_to_hub("squad_conflict_v2_under_150_with_substitution")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

In [40]:
import random
import numpy as np
from numpy.linalg import norm

def check_cosine_similarity(arr1, arr2, thres):
    cosine_similarity = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))
    return (cosine_similarity <= thres) and (cosine_similarity >= 0.6)
    
def find_non_identical_random_entity(ent_type, origin: str, thres: float=0.8):
    subsets = ent_to_answer_unique[ent_type]
    #random_entities = random.sample(subsets, 100 if len(subsets)>100 else len(subsets))
    for entity in subsets:#random_entities:
        origin_vec, entity_vec = entity_vector_map[origin], entity_vector_map[entity]
        if check_cosine_similarity(origin_vec, entity_vec, thres):
            return entity
    return random.choice(subsets)

def find_random_entity(ent_type, origin: str):
    subsets = ent_to_answer_unique[ent_type]
    random_entity = random.choice(subsets)
    while random_entity == origin:
        random_entity = random.choice(subsets)
    return random_entity

In [420]:
for ent in random.sample(unique_entities, 50):
   print(f'[{mention_to_ner[ent]}] {ent} -> {find_non_identical_random_entity(mention_to_ner[ent], ent)}')

[QUANTITY] 800 or 1,000 ft -> 452.8 ft
[NORP] West Flemish -> East Slavic
[DATE] Jurassic-Cretaceous extinction -> 19th-century
[PERSON] Zane Lowe -> RANDOM---the Abbot Suger
[CARDINAL] 3,507 -> 70,000
[CARDINAL] one fifth -> one hundred and fifty-seven
[FAC] the Palazzo Pubblico -> the Theatre Royal
[FAC] the Henry Cole Wing -> the Collegium Maius
[PERCENT] 44.72% -> as low as 10%
[NORP] Romanesque -> Medieval
[PRODUCT] English Oak -> RANDOM---Zamak
[GPE] East Prussia -> RANDOM---Sankt Goarshausen
[DATE] December 1994 -> December 25, 2011
[PERSON] James Green -> Dudley Simpson
[CARDINAL] 643 million -> two million
[PERSON] Charles IV -> RANDOM---Cannon
[PERCENT] 2.65% -> Forty percent
[PERSON] Kristjan Jaak Peterson -> Alvaro Martin
[DATE] July 14, 2008 -> between 14 to 17 years of age
[LOC] Cape Guardafui -> RANDOM---the Indian Ocean
[ORG] NHL -> RANDOM---National Islamic Front
[DATE] between 333 and 390 -> between 2001 and 2004
[PERSON] Thoraya Ahmed Obaid -> Hussein Sirri Amer
[WOR

In [427]:
for ent in random.sample(unique_entities, 50):
   print(f'[{mention_to_ner[ent]}] {ent} -> {find_random_entity(mention_to_ner[ent], ent)}')

[EVENT] Henley Royal Regatta -> Super Bowl XLVIII
[PERSON] Ho Keung Tse -> John XXIII
[PERSON] Maria Skłodowska-Curie -> Arno Tausch
[NORP] Saxon -> Mestizo
[PERSON] Geoff Travis -> Mars
[FAC] Avenue J -> Boulogne-sur-Mer Harbour
[CARDINAL] 220 million -> 17 million
[DATE] 45 BC -> 1849
[MONEY] $25.6 billion -> $5.7 billion
[NORP] Mohism -> Freistaaten
[PRODUCT] Amiibo -> iMac G3
[DATE] between 1921 and 1923 -> Between 1836 and 1842
[PERSON] Lionel Logue -> Elbridge Gerry
[ORG] Comics Code Authority -> Arnold, Constable
[PERSON] Jonas Bronck -> Tom Benson
[PERSON] Bobby Vinton -> Richard Hoggart
[MONEY] $388 billion -> ₹1595 billion
[PERSON] Hans Hillerbrand -> Nikolai Diletsky
[PERSON] Anthony France -> Konstantin Chernenko
[DATE] around 6000 BC -> 7 July
[ORG] General Atomics -> LPGA
[PERSON] John Paul Weier -> Amartya Sen
[LOC] Sacramento Pass -> South-Western Ontario
[LAW] the Defence of the Realm Act -> the Unfair Commercial Practices Directive
[DATE] 910 -> October 11
[PERCENT] 4

In [1]:
from src.classes.qadataset import QADataset
dataset = QADataset.load("HF_SquadDataset_all")

Read 98169 examples from /data/seongil/datasets/normalized/HF_SquadDataset_all.jsonl.gz


In [3]:
dataset._report_dataset_stats()

Dataset Statistics
-------------------------------------------
Total Examples = 98169
Answer Type: PERSON | Size of Group: 5558
Answer Type: None | Size of Group: 76845
Answer Type: NUMERIC | Size of Group: 4428
Answer Type: DATE | Size of Group: 3902
Answer Type: LOCATION | Size of Group: 3779
Answer Type: ORGANIZATION | Size of Group: 3657
-------------------------------------------


In [5]:
from collections import defaultdict

grouped_examples = defaultdict(list)
for ex in dataset.examples:
    grouped_examples[ex.get_example_answer_type()].append(ex)

print("Dataset Statistics")
print("-------------------------------------------")
print(f"Total Examples = {len(dataset.examples)}")
for group, ex_list in grouped_examples.items():
    print(f"Answer Type: {group} | Size of Group: {len(ex_list)}")
print("-------------------------------------------")

Dataset Statistics
-------------------------------------------
Total Examples = 98169
Answer Type: PERSON | Size of Group: 5558
Answer Type: None | Size of Group: 76845
Answer Type: NUMERIC | Size of Group: 4428
Answer Type: DATE | Size of Group: 3902
Answer Type: LOCATION | Size of Group: 3779
Answer Type: ORGANIZATION | Size of Group: 3657
-------------------------------------------


In [ ]:
grouped_examples["PERSON"]